In [1]:
#путь к данным
path = './data/server3.lpm.org.ru/~baulin/mmCIF/out/'

# Получаем список нужных файлов и цепей из NR-списка

In [2]:
#NR-лист
filter_data = open('./data/nrlist_3.36_all.csv')

filter_outfile = [] 
for i, line in enumerate(filter_data):
    class_ = line.split('\",\"')[1].split('+')
    filter_outfile += class_

filter_data.close()


In [3]:
files_chains = {}
for class_ in filter_outfile:
    class_ = class_.split('|')
    file_name = class_[0].lower() + '.out' + class_[1].strip()
    if file_name not in files_chains:
        files_chains[file_name] = []
    files_chains[file_name].append(class_[2])

In [4]:
#смотрим, норм ли напарсили
for i, x in enumerate(files_chains):
    print x, files_chains[x]
    if i>5:
        break

2mfc.out1 ['B']
1zig.out1 ['A']
4k4s.out1 ['B', 'F', 'C', 'G', 'D', 'H']
5dno.out1 ['B']
5x3z.out1 ['B']
3cjz.out1 ['A', 'B']
1arj.out7 ['N']


# Строим набор компонент связности по файлу

In [5]:
#пример данных: файл + список цепей

data = open(path +'4v6u.out1').read()
chains =  ['A2', 'A0', 'A1', 'B3', 'B1']

In [6]:
import re
stem_re  = r'List of (\d+) stems'
stack_re = r'List of (\d+) stacks'
neighbor_re = r'Summary of structural features of (\d+) nucleotides'

In [7]:
#Достаем ребра из блока 'List of (\d+) stems'
m_stem = re.search(stem_re, data)
data_stems = data[m_stem.end():].split('****************************************************************************')[0]
stems = data_stems.split('--------------------------------------------------------------------------')
stems_output = []
for stem in stems:
    stack1 = []
    stack2 = []
    for line in stem.split('\n'):
        line = line.split()
        if len(line) > 0:
            if line[0].isdigit():
                stack1.append(line[1])
                stack2.append(line[2])
    for i in range(len(stack1)-1):
        stems_output.append({'edge':(stack1[i], stack1[i+1]), 'type':'stem'} )
    for i in range(len(stack2)-1):
        stems_output.append({'edge':(stack2[i], stack2[i+1]), 'type':'stem'})

In [8]:
#Достаем ребра из блока 'List of (\d+) stacks'
m_stack = re.search(stack_re, data)
data_stacks = data[m_stack.end():].split('****************************************************************************')[0]
stacks_output = []
for stack in data_stacks.split('\n'):
    stack = stack.strip().split()
    if len(stack)>0:
        if stack[0].isdigit():
            stack = stack[3].split(',')
            for i in range(len(stack)-1):
                stacks_output.append({'edge':(stack[i], stack[i+1]), 'type':'stack'} )

In [9]:
#Достаем ребра из блока 'Summary of structural features of (\d+) nucleotides'
m_neighbor = re.search(neighbor_re, data) 
data_neighbors = data[m_neighbor.end():].split('****************************************************************************')[0]
neighbor_output = []
last_item = None
for stack in data_neighbors.split('\n'):
    stack = stack.strip().split()
    if len(stack)>0:
        if stack[0].isdigit():
            if last_item is None:
                last_item = stack[3]
            else:
                if last_item.split('.')[:-3]==stack[3].split('.')[:-3]:
                    neighbor_output.append({'edge':( last_item, stack[3]), 'type':'neighbor'} )
                    last_item = stack[3]
                else:
                    last_item = stack[3]

In [10]:
#Все ребра между нуклеотидами (без связей внутри А-минора)
edges = stacks_output+stems_output + neighbor_output

In [11]:
#Достаем А-миноры 'List of (\d+) A-minor motifs'
a_minor_re = r'List of (\d+) A-minor motifs'
m_a_minor = re.search(a_minor_re, data) 
a_minor_data = data[m_a_minor.end():].split('****************************************************************************')[0]

a_minors = {}
a_minor_vert = set()
last_item = None
for stack in a_minor_data.split('\n'):
    stack = stack.strip().split()
    if len(stack)>0:
        if stack[0].isdigit():
            if stack[3].split('.')[2] in chains:
                a_minor_vert.add(stack[3])
            f_stack, mid_stack = stack[3].split('|')
            s_stack, th_stack  = mid_stack.split(',')
            if f_stack not in a_minors:
                a_minors[f_stack] = []
            a_minors[f_stack].append(stack[3])
            if s_stack not in a_minors:
                a_minors[s_stack] = []
            a_minors[s_stack].append(stack[3])
            if th_stack not in a_minors:
                a_minors[th_stack] = []
            a_minors[th_stack].append(stack[3])

In [12]:
#Матрица смежности А-миноров

matrix = {}
for i, x in enumerate(a_minor_vert):
    matrix[x]={}
    for y in a_minor_vert:
        if x<y:
            matrix[x][y]=[]

In [13]:
#связь между А-минорами по общим нуклеотидам
for c in a_minors:
    if len(a_minors[c])>1:
        for i in range(len(a_minors[c])):
            for j in range(len(a_minors[c])):
                if a_minors[c][i]<a_minors[c][j]:
                    matrix[a_minors[c][i]][a_minors[c][j]].append('a_minor_'+c)
print len(a_minors)

534


In [14]:
for e in edges:
    a, b = min(e['edge']), max(e['edge'])
    if a in a_minors and b in a_minors:
        for x in a_minors[a]:
            for y in a_minors[b]:
                if x < y:
                    matrix[x][y].append(e['type']+'_'+a+'_'+b)
                if x>y:
                    matrix[y][x].append(e['type']+'_'+a+'_'+b)

In [15]:
for x in matrix:
    for y in matrix:
        if x < y:
            matrix[y][x]=matrix[x][y]
    matrix[x][x]=[]

# Связность компонент

In [16]:

def sub_dfs(u, matrix, visited, component):
    visited[u] = True
    for v in matrix[u]:
        if len(matrix[u][v])>0:
            component.append({'edge': (u, v), 'links': matrix[u][v]})
            if not visited[v]:
                sub_dfs(v, matrix, visited, component)

def dfs(matrix):
    text=[]
    components = []
    visited = {ver:False for ver in matrix}
    component = []
    for v in matrix:            
        if not visited[v]: 
            if len(component) > 0:
                components.append(component)
            component = []
            sub_dfs(v, matrix, visited, component)
    for x in components:
        text.append(str(x))
    return '\n'.join(text)
    

In [18]:
f = open('data_sample_1', 'w')
f.write(dfs(matrix))
f.close()